# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [55]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [56]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sisimiut,66.9395,-53.6735,-22.22,92,100,1.48,GL,1738804537
1,1,coronel suarez,-37.4547,-61.9334,16.64,93,12,2.75,AR,1738804539
2,2,boa vista,2.8197,-60.6733,28.99,65,75,5.14,BR,1738804488
3,3,grytviken,-54.2811,-36.5092,1.78,95,100,3.82,GS,1738804541
4,4,port-aux-francais,-49.3500,70.2167,7.42,94,100,9.41,TF,1738804542


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [57]:
%%capture --no-display

city_map = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size='Humidity', 
    color='City',
    alpha=0.7,
    tiles='OSM',  # Optional: Use OpenStreetMap as the base layer
    title="City Map with Humidity Represented by Point Size"
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [58]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 15) &
    (city_data_df['Max Temp'] <= 25) &
    (city_data_df['Humidity'] < 60) &
    (city_data_df['Cloudiness'] < 10)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,79,maiyama,12.0841,4.3677,23.03,9,1,4.81,NG,1738804628
89,89,faya-largeau,17.9257,19.1043,19.54,22,0,10.99,TD,1738804639
129,129,zouerate,22.7187,-12.4521,15.64,25,0,3.02,MR,1738804684
175,175,margaret river,-33.9500,115.0667,22.92,59,0,6.94,AU,1738804734
194,194,rada tilly,-45.9333,-67.5333,21.27,30,0,9.26,AR,1738804756


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
79,maiyama,NG,12.0841,4.3677,9,
89,faya-largeau,TD,17.9257,19.1043,22,
129,zouerate,MR,22.7187,-12.4521,25,
175,margaret river,AU,-33.9500,115.0667,59,
194,rada tilly,AR,-45.9333,-67.5333,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [60]:
# Set parameters to search for a hotel
import requests

# Set parameters to search for a hotel
radius = 10000  # Radius in meters
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key  # Only get the first result
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    name_address = response.json()
    # Convert the API response to JSON format
    try:
        # name_address = response.json()
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
maiyama - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
rada tilly - nearest hotel: No hotel found
socorro - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
sao raimundo nonato - nearest hotel: No hotel found
rio gallegos - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
santa luzia - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
puerto palomas - nearest hotel: No hotel found
atar - nearest hotel: No hotel found
la tijera - nearest hotel: No hotel found
tionk essil - nearest hotel: No hotel found
pitiquito - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
miguel ahumada - nearest hotel: No hotel found
viedma - nearest hotel: No hotel found
pasni - nearest hotel: No hotel found
beinamar - nearest hotel: No hotel found
ratnagiri - nearest hotel: No hotel f

,City,Country,Lat,Lng,Humidity,Hotel Name
79,maiyama,NG,12.0841,4.3677,9,No hotel found
89,faya-largeau,TD,17.9257,19.1043,22,No hotel found
129,zouerate,MR,22.7187,-12.4521,25,No hotel found
175,margaret river,AU,-33.9500,115.0667,59,No hotel found
194,rada tilly,AR,-45.9333,-67.5333,30,No hotel found
207,socorro,US,31.6546,-106.3033,10,No hotel found
246,cabo san lucas,MX,22.8909,-109.9124,28,No hotel found
256,sao raimundo nonato,BR,-9.0153,-42.6994,59,No hotel found
258,rio gallegos,AR,-51.6226,-69.2181,47,No hotel found
280,codrington,AU,-38.2667,141.9667,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [61]:
%%capture --no-display

city_map = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size='Humidity', 
    color='City',
    alpha=0.7,
    tiles='OSM',
    hover_cols=['City', 'Country', 'Hotel Name'],  # Add hover information
    title="City Map with Hotel Names and Countries in Hover"
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)